In [26]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
#read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [28]:
len(words)

32033

In [29]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [30]:
# make the dataset

block_size = 3
X, Y = [], []
for w in words:
    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        #print(''.join(itos[a] for a in context), '--->', itos[ix])
        context = context[1:] + [ix]
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [31]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182568, 3]) torch.Size([182568])
torch.Size([22696, 3]) torch.Size([22696])
torch.Size([22882, 3]) torch.Size([22882])


In [32]:
Xtr.shape, Ytr.shape # dataset

(torch.Size([182568, 3]), torch.Size([182568]))

In [ ]:
embd = 10
hls = 200
C = torch.randn((27, embd))
W1 = torch.randn((embd * block_size, hls)) * (5/3)/(embd * block_size)**.5
#b1 = torch.randn(hls) * 0
W2 = torch.randn((hls, 27)) * 0.01 #want to make very small
b2 = torch.randn(27) * 0
bngain = torch.ones((1, hls))
bnbias = torch.zeros((1, hls))
bnmean_running = torch.zeros((1, hls))
bnstd_running = torch.ones((1, hls))
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [34]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
lri = []
lossi = []
stepi = []

for i in range(200000):
    #minibatch
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    #forward pass
    emb = C[Xtr[ix]]
    embcat = emb.view(-1, embd * block_size)
    hpreact = embcat @ W1 #+b1
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
    with torch.no_grad():
        bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
        bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi

    h = torch.tanh(hpreact)
    logits = h @ W2 + b2
    logits.shape
     #counts = logits.exp()
    #prob = counts / counts.sum(1, keepdims=True)
    #loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix])
    #print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    #lr = lrs[i]
    lr = 0.1
    
    if i > 100000:
        lr = 0.01
    
    if i > 200000:
        lr = 0.001
    
    for p in parameters:
        p.data += -lr * p.grad
        
    if i % 10000 == 0:
        print(f'{i:7d}/ 300000: {loss.item():.4f}')
        
    #track stats
    #lri.append(lre[i])
    stepi.append(i)
    lossi.append(loss.log10().item())


      0/ 300000: 3.2996
  10000/ 300000: 2.5669
  20000/ 300000: 2.5009
  30000/ 300000: 2.3827
  40000/ 300000: 2.2899
  50000/ 300000: 2.2380
  60000/ 300000: 2.0132
  70000/ 300000: 2.0373
  80000/ 300000: 2.1768
  90000/ 300000: 2.4063
 100000/ 300000: 2.1205
 110000/ 300000: 1.8886
 120000/ 300000: 2.1221
 130000/ 300000: 2.2904
 140000/ 300000: 2.5514
 150000/ 300000: 2.2734
 160000/ 300000: 1.8383
 170000/ 300000: 2.3766
 180000/ 300000: 2.0618
 190000/ 300000: 2.1551


In [36]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x] # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 # + b1
    #hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias
    hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 # (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0680696964263916
val 2.1261446475982666


In [37]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, embd * block_size) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

3.0831382274627686

In [38]:
emb = C[Xte]
h = torch.tanh(emb.view(-1, embd * block_size) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yte)
loss.item()

3.056509017944336

In [39]:
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
        emb = C[torch.tensor([context])] # (1,block_size,d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))
        

khazlookdle.
payrikshlyngstabnexxmylardencvleeboldllemmixvyriqam.
shaw.
vyrenasmphavyogly.
shlvirasmivid.
jocelfijahvin.
hux.
zleighn.
laklaikloudnshaviouslutziritrakyrickquggx.
charylahgl.
lisiqievyn.
mden.
xxahtanthl.
zaklbertyn.
nnexlikmylakzlavigh.
maxxlonevin.
navan.
arykkzardaviyahfle.
shlvmerlynnlehmoneliayalliahng.
rothslyngstuv.


In [40]:
-torch.tensor(1/27.0).log()

tensor(3.2958)